In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d


In [ ]:
PROJECT_ROOT = Path(__file__).resolve().parents[1]
DATA_DIR = PROJECT_ROOT / "data_processing" / "processed_data"
SAVE_DIR = PROJECT_ROOT / "figures" / "analysis"

In [ ]:
ends = pd.read_csv(DATA_DIR / "ends_processed.csv")

In [ ]:
pp_varient = {
    "PP": ends.loc[ends["PowerPlayBool"] == 1],
    "NO_PP": ends.loc[ends["PowerPlayBool"] == 0]
}

final = {}
for action, variant in pp_varient.items():
    out = {}
    for end in range(1, 9):
        vc = variant.loc[variant["EndID"] == end]["EndDiff"].value_counts().to_dict()
        n = len(variant.loc[variant["EndID"] == end])
        out[end] = {k: v / n for k, v in vc.items()} if n > 0 else {}
    final[action] = out

output_dir = SAVE_DIR / "end_frequency_combined"
os.makedirs(output_dir, exist_ok=True)

actions = ["PP", "NO_PP"]
colors = {"PP": "blue", "NO_PP": "red"}
offsets = {"PP": 0.2, "NO_PP": -0.2}

all_enddiffs = sorted(
    {ed for a in actions for e in final[a] for ed in final[a][e]}
)

for end in range(1, 9):
    fig, ax = plt.subplots(figsize=(9, 5))

    ymax = 0

    for action in actions:
        if end not in final[action] or len(final[action][end]) < 3:
            continue

        freqs = np.array([final[action][end].get(ed, 0) for ed in all_enddiffs])
        freqs_smooth = gaussian_filter1d(freqs, sigma=1.5)

        ymax = max(ymax, freqs.max())

        ax.bar(
            [ed + offsets[action] for ed in all_enddiffs],
            freqs,
            width=0.35,
            color=colors[action],
            alpha=0.4,
            label=f"{action} (freq)"
        )

        ax.plot(
            all_enddiffs,
            freqs_smooth,
            color=colors[action],
            linewidth=2,
            marker="o",
            label=f"{action} (smooth)"
        )

    ax.set_title(f"End {end}: End Difference Distribution")
    ax.set_xlabel("End Difference")
    ax.set_ylabel("Frequency")
    ax.set_xticks(all_enddiffs)
    ax.set_ylim(0, ymax + 0.05)
    ax.grid(True, axis="y", linestyle="--", alpha=0.5)
    ax.legend()

    save_path = os.path.join(output_dir, f"end_{end}_frequency_combined.png")
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close(fig)

    print(f"Saved combined plot for End {end} → {save_path}")


Saved combined plot for End 1 → /Users/brentkong/Documents/curling/figures/analysis/end_frequency_combined/end_1_frequency_combined.png
Saved combined plot for End 2 → /Users/brentkong/Documents/curling/figures/analysis/end_frequency_combined/end_2_frequency_combined.png
Saved combined plot for End 3 → /Users/brentkong/Documents/curling/figures/analysis/end_frequency_combined/end_3_frequency_combined.png
Saved combined plot for End 4 → /Users/brentkong/Documents/curling/figures/analysis/end_frequency_combined/end_4_frequency_combined.png
Saved combined plot for End 5 → /Users/brentkong/Documents/curling/figures/analysis/end_frequency_combined/end_5_frequency_combined.png
Saved combined plot for End 6 → /Users/brentkong/Documents/curling/figures/analysis/end_frequency_combined/end_6_frequency_combined.png
Saved combined plot for End 7 → /Users/brentkong/Documents/curling/figures/analysis/end_frequency_combined/end_7_frequency_combined.png
Saved combined plot for End 8 → /Users/brentkong